### Dynamic Selection with Cutom Selection Function

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination

import os

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
def weather_check_tool(city: str) -> str:
    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C",
        "Houston": "Sunny, 30°C",
        "Phoenix": "Hot, 35°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Cloudy, 20°C",
        "Berlin": "Sunny, 23°C",
    }

    return weather_data.get(city, "Weather data not available for this city.")


def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        "USD": {"EUR": 0.85, "JPY": 110.0},
        "EUR": {"USD": 1.18, "JPY": 130.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0077},
    }

    if from_currency in exchange_rates and to_currency in exchange_rates[from_currency]:
        rate = exchange_rates[from_currency][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is {converted_amount:.2f} {to_currency}."
    else:
        return "Currency conversion not available for the specified currencies."

In [4]:
model_client = OpenAIChatCompletionClient(
    # azure_deployment = os.getenv("azure_deployment"),
    # model = os.getenv("model_name"),
    # api_version = os.getenv("api_version"),
    # azure_endpoint=os.getenv("azure_endpoint"),
    # api_key=os.getenv("api_key")
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [5]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    model_client=model_client,
    system_message="""
        You are a planning agent that helps users with their tasks.
        It should break down tasks and delegate them to other appropriate agents.
        Your Team Members are:
        - WeatherAgent: Provides weather information for cities.
        - CurrencyAgent: Provides currency exchange rates and conversions.
        You only plan and delegate tasks, you do not execute them by yourself.
        When assigning tasks to agents, you should provide clear instructions in the following format.
        
        1. <agent> : <task>
        
        After all tasks are complete, summarize the findings and end with 'TERMINATE';
        """,
)

weather_agent = AssistantAgent(
    "WeatherAgent",
    model_client=model_client,
    system_message="""
        You are a weather agent that provides weather information for cities.
        When asked about the weather, respond with the current weather conditions for the specified city.
        If the city is not in your database, respond with 'Weather data not available for this city.'
        Your only tool is the weather_check_tool, which takes a city name as input and returns the weather information.
        """,
    tools=[weather_check_tool],
)

currency_agent = AssistantAgent(
    "CurrencyAgent",
    model_client=model_client,
    system_message="""
        You are a currency exchange agent that provides currency exchange rates and conversions.
        When asked about currency exchange, respond with the converted amount based on the provided exchange rates.
        If the currencies are not in your database, respond with 'Currency conversion not available for the specified currencies.'
        Your only tool is the currency_exchange_tool, which takes an amount, from_currency, and to_currency as input and returns the converted amount.
        """,
    tools=[currency_exchange_tool],
)

In [7]:
from typing import Sequence


def custom_selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name

    return None

In [9]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(5)
combined_termination = text_mention_termination & max_message_termination

team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    allow_repeated_speaker=True,
    selector_func=custom_selector_func,
)

In [11]:
task = "What's the current weather in New York and how much is 100 USD in EUR?"

await Console(
    team.run_stream(task=task))

---------- TextMessage (user) ----------
What's the current weather in New York and how much is 100 USD in EUR?
---------- TextMessage (PlanningAgent) ----------
1. WeatherAgent : Provide the current weather for New York.
2. CurrencyAgent : Convert 100 USD to EUR.
---------- ToolCallRequestEvent (WeatherAgent) ----------
[FunctionCall(id='call_HU8b30hos5apTeO0uS8H7jlQ', arguments='{"city":"New York"}', name='weather_check_tool')]
---------- ToolCallExecutionEvent (WeatherAgent) ----------
[FunctionExecutionResult(content='Sunny, 25°C', name='weather_check_tool', call_id='call_HU8b30hos5apTeO0uS8H7jlQ', is_error=False)]
---------- ToolCallSummaryMessage (WeatherAgent) ----------
Sunny, 25°C
---------- TextMessage (PlanningAgent) ----------
3. CurrencyAgent : Providing the exchange rate and conversion of 100 USD to EUR.
---------- ToolCallRequestEvent (CurrencyAgent) ----------
[FunctionCall(id='call_C7vuU1LIYyqMYJPIS8ogKk6K', arguments='{"amount":100,"from_currency":"USD","to_currency":

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 6, 16, 4, 13, 316073, tzinfo=datetime.timezone.utc), content="What's the current weather in New York and how much is 100 USD in EUR?", type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=25), metadata={}, created_at=datetime.datetime(2025, 6, 6, 16, 4, 21, 499684, tzinfo=datetime.timezone.utc), content='1. WeatherAgent : Provide the current weather for New York.\n2. CurrencyAgent : Convert 100 USD to EUR.', type='TextMessage'), ToolCallRequestEvent(source='WeatherAgent', models_usage=RequestUsage(prompt_tokens=171, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 6, 6, 16, 4, 28, 945180, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_HU8b30hos5apTeO0uS8H7jlQ', arguments='{"city":"New York"}', name='weather_check_tool')], type='ToolCallRequestEvent'), ToolCallExecutio